# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!
model prediksi berbentuk diagram pohon yang digunakan dalam machine learning

# Import Data & Libraries

In [2]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [5]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    return set([row[col] for row in rows])

# contoh penggunaan
print(unique_vals(training_data, 0))
print(unique_vals(training_data, 1))

{'Yellow', 'Green', 'Red'}
{1, 3}


In [6]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
print(class_counts(training_data))

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [7]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# contoh penggunaan
print(is_numeric(7))
print(is_numeric("Red"))

True
False


In [8]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        # Bandingkan nilai fitur di kolom pertanyaan dengan nilai pertanyaan.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        # Ini hanya helper untuk mencetak pertanyaan dalam format yang mudah dibaca.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
# Mari kita buat pertanyaan untuk mencocokkan baris pertama di data pelatihan.
# Apakah warnanya Hijau?
q = Question(0, 'Green')
example = training_data[0]
print(q.match(example))

# contoh penggunaan 2
# Apakah diameternya >= 3?
q = Question(1, 3)
example = training_data[0]
print(q.match(example))

True
True


In [9]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
# Mari kita pisahkan data pelatihan berdasarkan pertanyaan.
# Apakah warnanya Merah?
q = Question(0, 'Red')
true_rows, false_rows = partition(training_data, q)
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [10]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# contoh penggunaan
print(gini(training_data))

0.6399999999999999


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [11]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
# Hitung Gini Impurity untuk seluruh dataset
current_uncertainty = gini(training_data)

# Bagaimana Information Gain dari membagi berdasarkan warna?
# Apakah warnanya Hijau?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
print(info_gain(true_rows, false_rows, current_uncertainty))

# Bagaimana Information Gain dari membagi berdasarkan diameter?
# Apakah diameternya >= 3?
true_rows, false_rows = partition(training_data, Question(1, 3))
print(info_gain(true_rows, false_rows, current_uncertainty))

0.1399999999999999
0.37333333333333324


In [12]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print(best_gain)
print(best_question)

0.37333333333333324
Is diameter >= 3?


# Fungsi Decision Tree

In [13]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.counts = class_counts(rows)

In [14]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
# membangun decision tree secara rekursif
def build_tree(rows):
    # Mencoba membagi dataset di setiap fitur
    # mencari pertanyaan terbaik dengan information gain tertinggi.
    gain, question = find_best_split(rows)

    # Base case: Tidak ada information gain
    # Ini berarti kita sudah di node daun.
    if gain == 0:
        return Leaf(rows)

    # Jika ada information gain, bagi dataset berdasarkan pertanyaan terbaik.
    true_rows, false_rows = partition(rows, question)

    # Rekursif membangun cabang true.
    true_branch = build_tree(true_rows)

    # Rekursif membangun cabang false.
    false_branch = build_tree(false_rows)

    # Mengembalikan node keputusan.
    return Decision_Node(question, true_branch, false_branch)

In [16]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.counts)
        return

    # mencetak pertanyaan pada node saat ini
    print (spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

# contoh penggunaan
my_tree = build_tree(training_data)
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [17]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.counts

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

# contoh penggunaan
# Klasifikasikan baris pertama dari data pengujian
print(classify(testing_data[0], my_tree))

{'Apple': 1}


In [18]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values())
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

# contoh penggunaan
print(print_leaf(classify(testing_data[0], my_tree)))
print(print_leaf(classify(testing_data[1], my_tree)))

{'Apple': '100%'}
{'Apple': '50%', 'Lemon': '50%'}


# Predict Using Decision Tree

In [19]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
for row in testing_data:
    print("Actual: %s. Predicted: %s" %
          (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
